In [15]:
import torch
import torch.nn as nn


config = {
    'train_file_path': 'dataset/train.csv',
    'test_file_path': 'dataset/test.csv',
    'embedding_path': 'dataset/sgns.weibo.word.bz2',
    'train_val_ratio': 0.1,
    'vocab_size': 30000,
    'batch_size': 64,
    'num_epochs': 10,
    'learning_rate': 1e-3,
    'logging_step': 300,
    'seed': 10003
}

config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

10003

In [7]:
# train.csv 4 cols: id, label, label_desc, sentence
from collections import Counter
from tqdm import tqdm
import jieba

def get_vocab(config):
    token_counter = Counter()
    with open(config['train_file_path'], 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in tqdm(lines, desc='Counting tokens', total=len(lines)):
            sent = line.split(',')[-1].strip()
            sent_cut = list(jieba.cut(sent))
            token_counter.update(sent_cut)
            # token_counter {'我': 2, '是': 5, ...}
    
    vocab = set(token for token, _ in token_counter.most_common(config['vocab_size']))
    return vocab

In [9]:
vocab = get_vocab(config)

Counting tokens: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53361/53361 [00:07<00:00, 6960.64it/s]


In [12]:
import bz2

with bz2.open(config['embedding_path']) as f:
    token_vector = f.readlines()

In [14]:
for i, line in enumerate(token_vector):
    if i == 10:
        line = line.split()
        print(line[0].decode('utf-8'))
        # print(line[1:])
        print(len(line[1:]))
        break

是
300


In [18]:
# 将 词典(vocab) 中的token 转化为 词向量
# token -> embedding
# token -> id

# '是' <-> 10 <-> 300d vector

def get_embedding(vocab):
    token2embedding = {}
    
    with bz2.open('dataset/sgns.weibo.word.bz2') as f:
        token_vector = f.readlines()
        
        meta_info = token_vector[0].split()
        print(f'{int(meta_info[0])} tokens in embedding file in total, vector size is {int(meta_info[1])}')
        
        # sgns.weibo.word.bz2 从第二行开始, 每一行是 'token embedding' 的形式
        # '我' 0.88383 .... (300)
        for line in tqdm(token_vector[1:]):
            line = line.split()
            token = line[0].decode('utf-8')
            
            vector = line[1:]
            
            if token in vocab:
                token2embedding[token] = [float(num) for num in vector]
        
        # enumerate(, [start])
        token2id = {token: idx for idx, token in enumerate(token2embedding.keys(), 4)}
        id2embedding = {token2id[token]: embedding for token, embedding in token2embedding.items()}
        
        PAD, UNK, BOS, EOS = '<pad>', '<unk>', '<bos>', 'eos'
        
        token2id[PAD] = 0
        token2id[UNK] = 1
        token2id[BOS] = 2
        token2id[EOS] = 3
        
        id2embedding[0] = [.0] * int(meta_info[1])
        id2embedding[1] = [.0] * int(meta_info[1])
        id2embedding[2] = np.random.random(int(meta_info[1])).tolist()
        id2embedding[3] = np.random.random(int(meta_info[1])).tolist()
        
        emb_mat = [id2embedding[idx] for idx in range(len(id2embedding))]
        
        return torch.tensor(emb_mat, dtype=torch.long), token2id, len(vocab) + 4

In [ ]:
emb_mat, token2id, config['vocab_size'] = get_embedding(vocab)
print(token2id)